In [4]:
import torch
import data_loader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import json
from collections import defaultdict

In [5]:
torch.cuda.is_available()

True

In [15]:
# trainx, devx, testx, trainy, devy, testy = data_loader.load_all_classic_random_split(flatten=False)
trainx, devx, testx, trainy, devy, testy = data_loader.load_all_subject_split(flatten=False)

train_subjects ['kevin_11_7', 'russell_11_20_stand', 'kelly_11_7', 'zhaoye', 'albert', 'russell_random_12_7', 'yiheng_11_30', 'joanne', 'solomon', 'jq_12_6', 'yongxu_11_30', 'canon_12_5', 'yiheng_12_5', 'chen', 'daniel', 'isa_12_5', 'ruocheng']
dev_subjects ['wenzhou_12_5', 'haobin_11_22']
test_subjects ['janet', 'russell_11_7']
Processing kevin_11_7
Processing russell_11_20_stand
Processing kelly_11_7
Processing zhaoye
Processing albert
Processing russell_random_12_7
Processing yiheng_11_30
Processing joanne
Processing solomon
Processing jq_12_6
Processing yongxu_11_30
Processing canon_12_5
Processing yiheng_12_5
Processing chen
Processing daniel
Processing isa_12_5
Processing ruocheng
Processing wenzhou_12_5
Processing haobin_11_22
Processing janet
Processing russell_11_7


In [16]:
# trainx, trainy = data_loader.augment_train_set(trainx, trainy, augment_prop=4, is_flattened=False)
trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape

((8362, 100, 3), (1039, 100, 3), (1000, 100, 3), (8362,), (1039,), (1000,))

In [17]:
BATCH_SIZE = 500

def get_dataloader(x, y, batch_size):
    dataset = [(x[i].T, y[i]) for i in range(y.shape[0])]
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

trainloader = get_dataloader(trainx, trainy, BATCH_SIZE)
devloader = get_dataloader(devx, devy, BATCH_SIZE)
testloader = get_dataloader(testx, testy, BATCH_SIZE)

In [18]:
_, num_feature, num_channel = trainx.shape
num_feature, num_channel

(100, 3)

In [19]:
def acc_loss(data_loader, criterion):
    correct = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
                
            outputs = net(x.float())
            _, predicted = torch.max(outputs.data, 1)
            
            w = torch.sum((predicted - y) != 0).item()
            r = len(y) - w
            correct += r 
            total += len(y)
            
            total_loss += criterion(outputs, y.long()).item() * len(x)
    return correct / total, total_loss / total

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1
        )  
        # 16 channel, num_feature
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        # 16 channel, num_feature

        self.conv2 = nn.Conv1d(
            in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2
        )
        # 32 channel, num_feature
        self.pool2 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)
        # 32 channel, num_feature
        
        self.conv3 = nn.Conv1d(
            in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2
        )
        self.pool3 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)

        self.fc1 = nn.Linear(num_feature * 64, 3200)
        self.fc2 = nn.Linear(3200, 1600)
        self.fc3 = nn.Linear(1600, 500)
        self.out = nn.Linear(500, 26)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, num_feature * 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x
net = Net()
if torch.cuda.is_available():
    net.cuda()
net

Net(
  (conv1): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool2): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool3): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6400, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=1600, bias=True)
  (fc3): Linear(in_features=1600, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=26, bias=True)
)

In [21]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
# optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer = optim.AdamW(net.parameters(), weight_decay=0.1)

hist = defaultdict(list)
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
    trainacc, trainloss = acc_loss(trainloader, criterion)
    devacc, devloss = acc_loss(devloader, criterion)
    hist['trainacc'].append(trainacc)
    hist['trainloss'].append(trainloss)
    hist['devacc'].append(devacc)
    hist['devloss'].append(devloss)
    
    print(f'Epoch {epoch} trainacc={trainacc} devacc={devacc}')
    print(f'        trainloss={trainloss} devloss={devloss}')

print('Finished Training')

Epoch 0 trainacc=0.04448696484094714 devacc=0.03849855630413859
        trainloss=3.2563130540215948 devloss=3.258410598132562
Epoch 1 trainacc=0.05931595312126286 devacc=0.03657362848893166
        trainloss=3.201957697105362 devloss=3.2658143630959415
Epoch 2 trainacc=0.17424061229370963 devacc=0.06063522617901829
        trainloss=2.824591537150319 devloss=3.3294917846437366
Epoch 3 trainacc=0.3228892609423583 devacc=0.0933589990375361
        trainloss=2.27039274283215 devloss=3.44392831355362
Epoch 4 trainacc=0.4738100932791198 devacc=0.13378248315688163
        trainloss=1.7396511428523251 devloss=3.266101997098289
Epoch 5 trainacc=0.5977038985888543 devacc=0.17805582290664101
        trainloss=1.3386873571646325 devloss=3.259829125344696
Epoch 6 trainacc=0.683448935661325 devacc=0.18960538979788258
        trainloss=1.0481630873377885 devloss=3.532945864469071
Epoch 7 trainacc=0.757354699832576 devacc=0.16361886429258904
        trainloss=0.7757267905062849 devloss=4.05488193207

In [22]:
testacc, testloss = acc_loss(testloader, nn.CrossEntropyLoss())
testacc, testloss
hist['testacc'] = testacc
hist['testloss'] = testloss

In [23]:
hist

defaultdict(list,
            {'trainacc': [0.04448696484094714,
              0.05931595312126286,
              0.17424061229370963,
              0.3228892609423583,
              0.4738100932791198,
              0.5977038985888543,
              0.683448935661325,
              0.757354699832576,
              0.8292274575460417,
              0.8658215737861755,
              0.8708442956230567,
              0.9252571155226023,
              0.911504424778761,
              0.9508490791676633,
              0.9638842382205214,
              0.9734513274336283,
              0.9753647452762497,
              0.9765606314278881,
              0.9760822769672327,
              0.9770389858885434],
             'trainloss': [3.2563130540215948,
              3.201957697105362,
              2.824591537150319,
              2.27039274283215,
              1.7396511428523251,
              1.3386873571646325,
              1.0481630873377885,
              0.7757267905062849,
        

In [74]:
with open('cnn_hist_random.json', 'w') as f:
    json.dump(hist, f)